##### __Skill Level:__ <span style="color:blue">Intermediate</span>

# Creating a Chatbot for Structured Data using BigQuery to implement RAG

## Overview

Generative AI (GenAI) represents a transformative technology capable of producing human-like text, images, code, and various other types of content. While much of the focus has been on unstructured data—such as PDFs, text documents, image files, and websites—many GenAI implementations rely on a parameter called "top K." This algorithm retrieves only the highest-scoring pieces of content relevant to a user's query, which can be limiting. Users seeking insights from structured data formats like CSV and JSON often require access to all relevant occurrences, rather than just a subset.

In this tutorial, we use a RAG agent together with a VertexAI chat model gemini-1.5-pro to query the BigQuery table. A Retrieval Augmented Generation (RAG) agent is a key part of a RAG application that enhances the capabilities of the large language models (LLMs) by integrating external data retrieval. AI agents empower LLMs to interact with the world through actions and tools. The agent connects to BigQuery, runs the query, and then sends the results to the model. The model processes the data and generates the final output that it is displayed for the user.


## Prerequisites

In this tutorial, we will be using Google Gemini Pro 1.5, which does not require deployment. However, if you prefer to use a different model, you can select one from the Model Garden via the console. This will allow you to add the model to your registry, create an endpoint (or utilize an existing one), and deploy the model—all in a single step. Here is a link for more information: [model deployment](https://cloud.google.com/vertex-ai/docs/general/deployment).

Before we begin, you'll need to create a Vertex AI RAG Data Service Agent service account. To do this, go to the IAM section of the console. Ensure you check the box for "Include Google-provided role grant." If the role is not listed, click "Grant Access" and add "Vertex AI RAG Data Service Agent" as a role. Additionally, to create datasets and tables in BigQuery, the user must have the __bigquery.datasets.create__ and __bigquery.tables.create__ permissions.

We are using a e2-standard-4 (Efficient Instance: 4 vCPUs, 16 GB RAM) for this tutorial. 

## Learning objectives

In this tutorial you will learn about:
- How to set up a BigQuery dataset and a table.
- How to load data to a BigQuery table.
- How to use the langchain ChatVertexAI agent to extract information from the table. 


## Pricing

If you are following this tutorial in one sitting it will cost ~$0.80 . Completing the process in multiple sessions or using a method different from the tutorial may result in increased costs.

## Get Started

### Install Packages

In [1]:
!pip install google-cloud-bigquery
!pip install gcloud
!pip install langchain
!pip install -U langchain-community
!pip install -qU langchain_google_vertexai
!pip install pandas-gbq
!pip install sqlalchemy-bigquery

  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602982 sha256=0a9c19da69ae0ae80fb3170b51ddfb53038a35aefa651421dfba55b49c4e6771
  Stored in directory: /home/jupyter/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into accou

Set your project id, location, and bucket variables.

In [7]:
project_id='cit-cl-odss-ramsivakumar-df75'
location='us-east4'
bucket = 'big-query-test-rs'
dataset_name = 'big_query_dataset_rs'
table_name = "test-table"

Create a bucket where the data used for tutorial is going to be placed. It is important to keep in mind that the name of the bucket has to be unique.

In [3]:
# make bucket
!gsutil mb -l {location} gs://{bucket}

Creating gs://big-query-test-rs/...


Once the bucket is created, we need to access the CSV source file. In this tutorial, we transferred the data file to our Jupyter notebook by simply dragging and dropping it from my local folder. Next, we need to specify the bucket name and the path of the data source in order to upload the CSV file to the bucket. 

We are using the [Data Science Salaries](https://www.kaggle.com/datasets/ajjarvis/ds-salaries) dataset from kaggle for this tutorial.

In [5]:
!gsutil cp 'ds_salaries.csv' gs://{bucket}

Copying file://health_screening_data.csv [Content-Type=text/csv]...
- [1 files][  4.8 MiB/  4.8 MiB]                                                
Operation completed over 1 objects/4.8 MiB.                                      


### Create a database in BigQuery

The next step is to create a database in BigQuery. To accomplish this, we need to define a dataset_id and construct a dataset object that will be sent to the API for creation. Note that in BigQuery, a dataset is analogous to a database.

In [8]:
from google.cloud import bigquery

client = bigquery.Client()

# Set dataset_id to the ID of the dataset to create.
dataset_id = f'{project_id}.{dataset_name}'

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Specify the geographic location where the dataset should reside.
dataset.location = location

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset cit-cl-odss-ramsivakumar-df75.big_query_dataset_rs


Now you have two options to create your table (please pick one):

#### Option 1: Auto detect 

The following code will auto detect the table schema and upload the data to the table in one step!

In [10]:
table_id = f'{dataset_id}.{table_name}'

job_config = bigquery.LoadJobConfig(
    autodetect=True, source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1,
)
uri = f"gs://{bucket}/ds_salaries.csv"
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 69960 rows.


#### Option 2: Manually create schema and upload CSV file

Alternatively, if we choose to define the table schema and create the dataset table beforehand, we can use the following code. Afterward, we can upload the CSV dataset into a pandas DataFrame and then insert it into the newly created BigQuery table.

In [ ]:
# Set table_id to the ID of the table to create.

client = bigquery.Client()

table_id = f'{project_id}.{dataset_name}.{table_name}'

schema = [
    bigquery.SchemaField("id", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("age", "INTEGER", mode="NULLABLE"),    
    bigquery.SchemaField("gender", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("height", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("weight", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ap_hi", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("ap_lo", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("cholesterol", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("gluc", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("smoke", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("alco", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("active", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("cardio", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("ageinyr", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("bmi", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("bmicat", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("agegroup", "STRING", mode="NULLABLE"),

]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

We load our dataset located in our bucket to a pandas dataframe.

In [ ]:
from google.cloud import storage
import pandas as pd
from io import StringIO

path = "<gsutil URI>"
df = pd.read_csv(path)
df.head()

We need to use the "to_gbq" function from the pandas-gbq library, which allows us to write a Pandas DataFrame to a Google BigQuery table. This enables us to populate our BigQuery table with the data from the DataFrame.

In [ ]:
df.to_gbq(table_id, project_id=project_id)

client.load_table_from_dataframe(df, table_id).result()

This step is optional. We can make an API request to verify whether the dataset has been successfully created under our project.

In [ ]:
datasets = list(client.list_datasets())  # Make an API request.
project = project_id

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("{} project does not contain any datasets.".format(project))

![image.png](../../images/gcp_rag_structure_data_01.png)

### Optional: Run a query

This step is also optional. We can execute a simple query on the BigQuery table to count the number of records.

In [13]:
query = f"""
    SELECT count(gender) as cgender
    FROM '{project_id.dataset_id.table_id}'
"""

# Execute the query
query_job = client.query(query)

# Fetch the results
results = query_job.result()

# Iterate through the rows
for row in results:
    #print(f"gender: {row.name}, age: {row.age}")
    print(f"cgender: {row.cgender}")

try:
    query_job = client.query(query)
    results = query_job.result()
    print(results)
except Exception as e:
    print(f"Error executing query: {str(e)}")

BadRequest: 400 Syntax error: Unexpected string literal 'project_id.dataset_id.table_id' at [3:10]; reason: invalidQuery, location: query, message: Syntax error: Unexpected string literal 'project_id.dataset_id.table_id' at [3:10]

Location: US
Job ID: d46a901a-4a22-4bb6-b499-0e32e78fc550


cgender: 139920
<google.cloud.bigquery.table.RowIterator object at 0x7f226b88ab90>

### Connect our model and Big Query database and run a query

To interact with the BigQuery table using a Pythonic domain language, we utilize SQLAlchemy. SQLAlchemy is a Python SQL toolkit that enables developers to access and manage SQL databases, allowing users to write queries as strings or chain Python objects for similar queries. 

In [14]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

sqlalchemy_url = f'bigquery://{project_id}/{dataset_name}'
print(sqlalchemy_url)

bigquery://cit-cl-odss-ramsivakumar-df75/big_query_dataset_rs


Next, we import the __ChatVertexAI__ agent from Langchain and configure it with the appropriate hyperparameters. In this instance, we are using the __gemini-1.5-pro__ LLM model, after which we create the SQL agent to enable querying the BigQuery table using a natural language string as a prompt. Temperature regulates randomness, with higher temperatures resulting in more varied and unpredictable outputs. Top-k sampling selects from the k most probable next tokens at each step, where a lower k emphasizes higher-probability tokens. The max tokens hyperparameter specifies the maximum number of tokens in the response from the large language model. Max retries indicates how many responses we will receive from the model.

In [16]:
from langchain_google_vertexai import VertexAI, ChatVertexAI

llm = ChatVertexAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=8190,
    max_retries=2,
)
db = SQLDatabase.from_uri(sqlalchemy_url)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=100000
    )


For our initial query, we check the number of rows to compare the result with what we obtained in a previous step using a simple SQL query against the BigQuery table. We can confirm that we received the same number of rows: 139,920.

In [17]:
agent_executor.run("How many rows are in the table screening? ")

/var/tmp/ipykernel_10383/3618013291.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("How many rows are in the table screening? ")




> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: test-tableAction: sql_db_schema
Action Input: test-table
CREATE TABLE `test-table` (
	`int64_field_0` INT64, 
	`id` INT64, 
	`age` INT64, 
	`gender` INT64, 
	`height` INT64, 
	`weight` FLOAT64, 
	`ap_hi` INT64, 
	`ap_lo` INT64, 
	`cholesterol` INT64, 
	`gluc` INT64, 
	`smoke` INT64, 
	`alco` INT64, 
	`active` INT64, 
	`cardio` INT64, 
	`AgeinYr` INT64, 
	`BMI` FLOAT64, 
	`BMICat` STRING, 
	`AgeGroup` STRING
)

/*
3 rows from test-table table:
int64_field_0	id	age	gender	height	weight	ap_hi	ap_lo	cholesterol	gluc	smoke	alco	active	cardio	AgeinYr	BMI	BMICat	AgeGroup
33817	48318	21582	2	178	11.0	130	90	1	1	0	0	1	1	59	3.5	Under Weight	40-60
57858	82567	18804	2	165	10.0	180	1100	2	2	0	0	1	1	51	3.7	Under Weight	40-60
29488	42156	20408	2	177	22.0	120	80	1	1	1	1	1	0	55	7.0

'The table test-table has 69960 rows.'

Next, we pose a question that requires applying a filter, and we successfully obtain the accurate number of obese individuals in the table.

In [ ]:
agent_executor.run("How many obese are in the table screen?")

As a more complex query, we inquired about the number of female smokers, and the SQL agent accurately returned the answer of 1,626 female smokers.

In [ ]:
agent_executor.run("How many smokers are female in the table screen? the value 1 in smoke means that is a smoker and the value 2 in gender means that is a female")

## Conclusion

You have learned how to create a dataset and a table in BigQuery, as well as how to set up an agent that enables you to query the table using natural language instead of SQL queries, allowing you to obtain the results you need.

## Clean Up

Please remember to delete or stop your Jupyter notebook and delete your BigQuery dataset and table to prevent incurring charges. And if you have created any other services like buckets, please remember to delete them as well.